## Doc2vec, Word2vec

### Useful resources
#### word2vec
https://thinkinfi.com/simple-doc2vec-explained/
https://thinkinfi.com/continuous-bag-of-words-cbow-multi-word-model-how-it-works/
https://www.youtube.com/watch?v=LSS_bos_TPI
https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469
https://www.youtube.com/watch?v=yFFp9RYpOb0

should do lots of cleaning task like stop word removal, replacing digits, remove punctuation etc.

#### doc2vec
We will use doc2vec

In [24]:
import numpy as np
import pandas as pd

essay_set = pd.read_csv(r"../../dataset/training_set_rel3.tsv", delimiter = '\t', header=0, encoding='latin1')
essay_set1 = pd.read_csv("../../clean_ver-1.csv")

train_corpus = essay_set1['essay']
test_corpus = train_corpus.copy()
train_corpus


0        Dear local newspaper, I think effects computer...
1        Dear @CAPS @CAPS, I believe that using compute...
2        Dear, @CAPS @CAPS @CAPS More and more people u...
3        Dear Local Newspaper, @CAPS I have found that ...
4        Dear @LOCATION, I know having computers has a ...
                               ...                        
12971     In most stories mothers and daughters are eit...
12972     I never understood the meaning laughter is th...
12973    When you laugh, is @CAPS out of habit, or is @...
12974                                   Trippin' on fen...
12975     Many people believe that laughter can improve...
Name: essay, Length: 12976, dtype: object

In [25]:
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer

# punctuation should be ignored and stop words
# preprocess text and compute counts
vocab = CountVectorizer(stop_words='english', lowercase= True).fit(train_corpus)

# generate counts for a new set of documents
doc_emb = vocab.transform(train_corpus)

In [26]:
#doc2vec tuning
vec_size = 50
window=2
min_count=1
workers=8
epochs=40

In [27]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


essays = [TaggedDocument(gensim.utils.simple_preprocess(doc), [i]) for i, doc in enumerate(train_corpus)]

model = Doc2Vec(essays, vector_size=vec_size, window=window, min_count=min_count, workers=workers, epochs=epochs)
#might not need this line
model.train(essays, total_examples=model.corpus_count, epochs=model.epochs)

In [28]:
doc = [gensim.utils.simple_preprocess(doc) for i, doc in enumerate(test_corpus)]

doc_emb = np.zeros((len(doc), vec_size))
for i in range(len(doc)):
    doc_emb[i,:] = model.infer_vector(doc[i])

In [29]:
sims = model.docvecs.most_similar([doc_emb[8094]]) #gives you top 10 document tags and their cosine similarity

In [6]:
# deltees training data
#model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [32]:
pd.DataFrame(doc_emb).to_csv("../../dataset/doc2vec_50.csv")